# Baseline Evaluation and Rotation Tightening

This notebook:
1. Evaluates the baseline submission score
2. Implements rotation tightening (fix_direction) optimization
3. Validates for overlaps

In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import math
import os

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]
TREE_VERTICES = np.array(list(zip(TX, TY)))

print("Tree vertices shape:", TREE_VERTICES.shape)
print("Tree height:", max(TY) - min(TY))
print("Tree width:", max(TX) - min(TX))

Tree vertices shape: (15, 2)
Tree height: 1.0
Tree width: 0.7


In [2]:
def parse_value(s):
    """Parse submission value (e.g., 's0.123' -> 0.123)"""
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def format_value(v):
    """Format value for submission (e.g., 0.123 -> 's0.123')"""
    return f's{v}'

def get_tree_polygon(x, y, deg):
    """Get the polygon for a tree at position (x,y) with rotation deg"""
    angle_rad = math.radians(deg)
    cos_a, sin_a = math.cos(angle_rad), math.sin(angle_rad)
    
    # Rotate and translate vertices
    rotated = []
    for vx, vy in TREE_VERTICES:
        rx = vx * cos_a - vy * sin_a + x
        ry = vx * sin_a + vy * cos_a + y
        rotated.append((rx, ry))
    
    return Polygon(rotated)

def load_submission(path):
    """Load submission CSV and parse values"""
    df = pd.read_csv(path)
    df['x_val'] = df['x'].apply(parse_value)
    df['y_val'] = df['y'].apply(parse_value)
    df['deg_val'] = df['deg'].apply(parse_value)
    return df

def get_trees_for_n(df, n):
    """Get all trees for n-tree configuration"""
    prefix = f'{n:03d}_'
    mask = df['id'].str.startswith(prefix)
    return df[mask].copy()

print("Utility functions defined")

Utility functions defined


In [3]:
def get_bounding_box_side(trees_df):
    """Calculate the side of the square bounding box for trees"""
    min_x, max_x = float('inf'), float('-inf')
    min_y, max_y = float('inf'), float('-inf')
    
    for _, row in trees_df.iterrows():
        poly = get_tree_polygon(row['x_val'], row['y_val'], row['deg_val'])
        bounds = poly.bounds  # (minx, miny, maxx, maxy)
        min_x = min(min_x, bounds[0])
        min_y = min(min_y, bounds[1])
        max_x = max(max_x, bounds[2])
        max_y = max(max_y, bounds[3])
    
    side = max(max_x - min_x, max_y - min_y)
    return side

def calculate_score(df):
    """Calculate the total score for a submission"""
    total_score = 0
    sides = {}
    
    for n in range(1, 201):
        trees = get_trees_for_n(df, n)
        if len(trees) == 0:
            print(f"Warning: No trees found for n={n}")
            continue
        
        side = get_bounding_box_side(trees)
        sides[n] = side
        total_score += (side ** 2) / n
    
    return total_score, sides

print("Scoring functions defined")

Scoring functions defined


In [4]:
# Load and evaluate baseline
baseline_path = '/home/code/experiments/001_baseline/baseline.csv'
df = load_submission(baseline_path)
print(f"Loaded {len(df)} rows")
print(f"Sample rows:")
print(df.head(10))

Loaded 20100 rows
Sample rows:
      id                     x                     y                  deg  \
0  001_0    s43.59119209210215   s-31.78326706874178   s44.99999999999998   
1  002_0   s0.1540970696213558  s-0.0385407426947946  s203.62937773065684   
2  002_1  s-0.1540970696213728   s-0.561459257305224  s23.629377730656792   
3  003_0   s1.1312705850687463   s0.7922028723269486  s113.56326044172948   
4  003_1     s1.23405569584216    s1.275999500663759     s66.370622269343   
5  003_2    s0.641714640229075    s1.180458566613381  s155.13405193710082   
6  004_0  s-0.3247477895908755   s0.1321099780911856   s156.3706221456364   
7  004_1   s0.3153543462411341   s0.1321099780664757   s156.3706222692641   
8  004_2   s0.3247477895908755  s-0.7321099780664752    s336.370622269264   
9  004_3  s-0.3153543481363216  s-0.7321099780911857  s336.37062214563645   

       x_val      y_val     deg_val  
0  43.591192 -31.783267   45.000000  
1   0.154097  -0.038541  203.629378  
2  -0.1

In [5]:
# Calculate baseline score
print("Calculating baseline score...")
score, sides = calculate_score(df)
print(f"\nBaseline Score: {score:.6f}")
print(f"\nTop 10 largest contributions (side^2/n):")
contributions = [(n, sides[n], (sides[n]**2)/n) for n in sides]
contributions.sort(key=lambda x: -x[2])
for n, side, contrib in contributions[:10]:
    print(f"  n={n}: side={side:.4f}, contribution={contrib:.4f}")

Calculating baseline score...



Baseline Score: 70.624424

Top 10 largest contributions (side^2/n):
  n=1: side=0.8132, contribution=0.6612
  n=2: side=0.9495, contribution=0.4508
  n=3: side=1.1420, contribution=0.4347
  n=5: side=1.4437, contribution=0.4168
  n=4: side=1.2908, contribution=0.4165
  n=7: side=1.6731, contribution=0.3999
  n=6: side=1.5484, contribution=0.3996
  n=9: side=1.8673, contribution=0.3874
  n=8: side=1.7559, contribution=0.3854
  n=15: side=2.3779, contribution=0.3769


In [6]:
def check_overlaps(trees_df):
    """Check for overlapping trees"""
    polygons = []
    for _, row in trees_df.iterrows():
        poly = get_tree_polygon(row['x_val'], row['y_val'], row['deg_val'])
        polygons.append(poly)
    
    if len(polygons) <= 1:
        return False, []
    
    tree_index = STRtree(polygons)
    overlaps = []
    
    for i, poly in enumerate(polygons):
        candidates = tree_index.query(poly)
        for j in candidates:
            if j > i:  # Only check each pair once
                if polygons[j].intersects(poly) and not polygons[j].touches(poly):
                    # Check if it's a real overlap (not just touching)
                    intersection = polygons[j].intersection(poly)
                    if intersection.area > 1e-10:
                        overlaps.append((i, j))
    
    return len(overlaps) > 0, overlaps

# Check a few configurations for overlaps
print("Checking for overlaps in sample configurations...")
for n in [10, 50, 100, 150, 200]:
    trees = get_trees_for_n(df, n)
    has_overlap, overlap_pairs = check_overlaps(trees)
    if has_overlap:
        print(f"  n={n}: OVERLAPS FOUND - {len(overlap_pairs)} pairs")
    else:
        print(f"  n={n}: OK")

Checking for overlaps in sample configurations...
  n=10: OK
  n=50: OK
  n=100: OK
  n=150: OK
  n=200: OK


In [7]:
def get_all_points(trees_df):
    """Get all polygon vertices for all trees"""
    all_points = []
    for _, row in trees_df.iterrows():
        poly = get_tree_polygon(row['x_val'], row['y_val'], row['deg_val'])
        coords = list(poly.exterior.coords)[:-1]  # Exclude closing point
        all_points.extend(coords)
    return np.array(all_points)

def rotate_points(points, angle_deg, center=None):
    """Rotate points around center by angle_deg degrees"""
    if center is None:
        center = points.mean(axis=0)
    
    angle_rad = math.radians(angle_deg)
    cos_a, sin_a = math.cos(angle_rad), math.sin(angle_rad)
    
    centered = points - center
    rotated = np.zeros_like(centered)
    rotated[:, 0] = centered[:, 0] * cos_a - centered[:, 1] * sin_a
    rotated[:, 1] = centered[:, 0] * sin_a + centered[:, 1] * cos_a
    
    return rotated + center

def get_bbox_side_for_points(points):
    """Get the side of the square bounding box for points"""
    min_x, max_x = points[:, 0].min(), points[:, 0].max()
    min_y, max_y = points[:, 1].min(), points[:, 1].max()
    return max(max_x - min_x, max_y - min_y)

def optimize_rotation_for_n(trees_df):
    """Find the optimal rotation angle to minimize bounding box"""
    all_points = get_all_points(trees_df)
    center = all_points.mean(axis=0)
    
    def objective(angle):
        rotated = rotate_points(all_points, angle, center)
        return get_bbox_side_for_points(rotated)
    
    # Try multiple starting points
    best_angle = 0
    best_side = objective(0)
    
    # Search in range [0, 90) since bounding box is symmetric
    result = minimize_scalar(objective, bounds=(0, 90), method='bounded')
    
    if result.fun < best_side:
        best_angle = result.x
        best_side = result.fun
    
    return best_angle, best_side

print("Rotation optimization functions defined")

Rotation optimization functions defined


In [8]:
def apply_rotation_to_trees(trees_df, angle_deg):
    """Apply rotation to all trees in a configuration"""
    if abs(angle_deg) < 0.001:
        return trees_df.copy()
    
    # Get center of all points
    all_points = get_all_points(trees_df)
    center = all_points.mean(axis=0)
    
    # Rotate each tree's position and add angle to rotation
    new_df = trees_df.copy()
    angle_rad = math.radians(angle_deg)
    cos_a, sin_a = math.cos(angle_rad), math.sin(angle_rad)
    
    for idx in new_df.index:
        x, y = new_df.loc[idx, 'x_val'], new_df.loc[idx, 'y_val']
        # Rotate position around center
        dx, dy = x - center[0], y - center[1]
        new_x = dx * cos_a - dy * sin_a + center[0]
        new_y = dx * sin_a + dy * cos_a + center[1]
        
        new_df.loc[idx, 'x_val'] = new_x
        new_df.loc[idx, 'y_val'] = new_y
        new_df.loc[idx, 'deg_val'] = (new_df.loc[idx, 'deg_val'] + angle_deg) % 360
    
    return new_df

# Test rotation optimization on a few configurations
print("Testing rotation optimization...")
for n in [5, 10, 20]:
    trees = get_trees_for_n(df, n)
    original_side = get_bounding_box_side(trees)
    best_angle, optimized_side = optimize_rotation_for_n(trees)
    improvement = original_side - optimized_side
    print(f"  n={n}: original={original_side:.6f}, optimized={optimized_side:.6f}, angle={best_angle:.2f}°, improvement={improvement:.6f}")

Testing rotation optimization...
  n=5: original=1.443692, optimized=1.443692, angle=0.00°, improvement=0.000000
  n=10: original=1.940696, optimized=1.940696, angle=0.00°, improvement=0.000000
  n=20: original=2.742469, optimized=2.742469, angle=0.00°, improvement=0.000000


In [9]:
# Apply rotation optimization to all configurations
print("Applying rotation optimization to all configurations...")

improved_df = df.copy()
total_improvement = 0
improvements = []

for n in range(1, 201):
    trees = get_trees_for_n(improved_df, n)
    original_side = get_bounding_box_side(trees)
    
    best_angle, optimized_side = optimize_rotation_for_n(trees)
    
    if optimized_side < original_side - 1e-8:
        # Apply the rotation
        rotated_trees = apply_rotation_to_trees(trees, best_angle)
        
        # Update the main dataframe
        for idx in trees.index:
            improved_df.loc[idx, 'x_val'] = rotated_trees.loc[idx, 'x_val']
            improved_df.loc[idx, 'y_val'] = rotated_trees.loc[idx, 'y_val']
            improved_df.loc[idx, 'deg_val'] = rotated_trees.loc[idx, 'deg_val']
        
        improvement = (original_side**2 - optimized_side**2) / n
        total_improvement += improvement
        improvements.append((n, original_side, optimized_side, best_angle, improvement))

print(f"\nTotal score improvement from rotation: {total_improvement:.6f}")
print(f"\nTop 10 improvements:")
improvements.sort(key=lambda x: -x[4])
for n, orig, opt, angle, imp in improvements[:10]:
    print(f"  n={n}: {orig:.6f} -> {opt:.6f} (angle={angle:.2f}°, improvement={imp:.6f})")

Applying rotation optimization to all configurations...



Total score improvement from rotation: 0.000004

Top 10 improvements:
  n=88: 5.535270 -> 5.535252 (angle=0.00°, improvement=0.000002)
  n=87: 5.535446 -> 5.535434 (angle=0.00°, improvement=0.000002)
  n=129: 6.636162 -> 6.636160 (angle=90.00°, improvement=0.000000)
  n=153: 7.172992 -> 7.172990 (angle=90.00°, improvement=0.000000)
  n=132: 6.637881 -> 6.637881 (angle=90.00°, improvement=0.000000)
  n=130: 6.636508 -> 6.636507 (angle=90.00°, improvement=0.000000)
  n=177: 7.715544 -> 7.715544 (angle=90.00°, improvement=0.000000)
  n=176: 7.715195 -> 7.715195 (angle=90.00°, improvement=0.000000)


In [10]:
# Calculate new score
print("Calculating improved score...")
new_score, new_sides = calculate_score(improved_df)
print(f"\nOriginal Score: {score:.6f}")
print(f"Improved Score: {new_score:.6f}")
print(f"Improvement: {score - new_score:.6f}")

Calculating improved score...



Original Score: 70.624424
Improved Score: 70.624421
Improvement: 0.000003


In [11]:
# Center all configurations to minimize coordinate values
def center_configuration(trees_df):
    """Center a configuration around origin"""
    all_points = get_all_points(trees_df)
    min_x, max_x = all_points[:, 0].min(), all_points[:, 0].max()
    min_y, max_y = all_points[:, 1].min(), all_points[:, 1].max()
    
    center_x = (min_x + max_x) / 2
    center_y = (min_y + max_y) / 2
    
    new_df = trees_df.copy()
    new_df['x_val'] = new_df['x_val'] - center_x
    new_df['y_val'] = new_df['y_val'] - center_y
    
    return new_df

# Center all configurations
print("Centering all configurations...")
for n in range(1, 201):
    trees = get_trees_for_n(improved_df, n)
    centered = center_configuration(trees)
    for idx in trees.index:
        improved_df.loc[idx, 'x_val'] = centered.loc[idx, 'x_val']
        improved_df.loc[idx, 'y_val'] = centered.loc[idx, 'y_val']

print("Done centering")

Centering all configurations...


Done centering


In [12]:
# Validate no overlaps were introduced
print("Validating no overlaps...")
has_any_overlap = False
for n in range(1, 201):
    trees = get_trees_for_n(improved_df, n)
    has_overlap, overlap_pairs = check_overlaps(trees)
    if has_overlap:
        print(f"  n={n}: OVERLAPS FOUND - {len(overlap_pairs)} pairs")
        has_any_overlap = True

if not has_any_overlap:
    print("All configurations valid - no overlaps!")

Validating no overlaps...


All configurations valid - no overlaps!


In [13]:
# Save improved submission
def save_submission(df, path):
    """Save submission in correct format"""
    output_df = pd.DataFrame({
        'id': df['id'],
        'x': df['x_val'].apply(format_value),
        'y': df['y_val'].apply(format_value),
        'deg': df['deg_val'].apply(format_value)
    })
    output_df.to_csv(path, index=False)
    print(f"Saved to {path}")

# Save to submission folder
submission_path = '/home/submission/submission.csv'
save_submission(improved_df, submission_path)

# Also save to experiment folder
experiment_path = '/home/code/experiments/001_baseline/submission.csv'
save_submission(improved_df, experiment_path)

print(f"\nFinal Score: {new_score:.6f}")

Saved to /home/submission/submission.csv
Saved to /home/code/experiments/001_baseline/submission.csv

Final Score: 70.624421


In [14]:
# Verify the saved submission
verify_df = load_submission(submission_path)
verify_score, _ = calculate_score(verify_df)
print(f"Verified Score: {verify_score:.6f}")

# Check first few rows
print("\nFirst 10 rows of submission:")
print(pd.read_csv(submission_path).head(10))

Verified Score: 70.624421

First 10 rows of submission:
      id                      x                      y                  deg
0  001_0   s0.15909902576697732  s-0.15909902576697377   s44.99999999999998
1  002_0   s0.15409706962136413    s0.2614592573052147  s203.62937773065684
2  002_1  s-0.15409706962136446   s-0.2614592573052147  s23.629377730656792
3  003_0    s0.2549376436978331  s-0.23343606154941587  s113.56326044172948
4  003_1    s0.3577227544712468    s0.2503605667873945     s66.370622269343
5  003_2  s-0.23461830114183824    s0.1548196327370166  s155.13405193710082
6  004_0   s-0.3247477888001553    s0.4321099780911857   s156.3706221456364
7  004_1   s0.31535434703185433    s0.4321099780664758   s156.3706222692641
8  004_2   s0.32474779038159574   s-0.4321099780664751    s336.370622269264
9  004_3   s-0.3153543473456014   s-0.4321099780911857  s336.37062214563645


In [15]:
# Check the improved submission from sa_fast_v2
sa_df = load_submission('/home/code/submission2.csv')
sa_score, sa_sides = calculate_score(sa_df)
print(f"SA Fast V2 Score: {sa_score:.6f}")
print(f"Improvement from baseline: {score - sa_score:.6f}")

SA Fast V2 Score: 70.621329
Improvement from baseline: 0.003095


In [17]:
# Check the second round of sa_fast_v2 optimization
sa_df2 = load_submission('/home/code/submission2.csv')
sa_score2, _ = calculate_score(sa_df2)
print(f"SA Fast V2 Round 2 Score: {sa_score2:.6f}")
print(f"Improvement from round 1: {sa_score - sa_score2:.6f}")
print(f"Total improvement from baseline: {score - sa_score2:.6f}")

SA Fast V2 Round 2 Score: 70.620680
Improvement from round 1: 0.000649
Total improvement from baseline: 0.003744


In [18]:
# Check the third round of sa_fast_v2 optimization
sa_df3 = load_submission('/home/code/submission2.csv')
sa_score3, _ = calculate_score(sa_df3)
print(f"SA Fast V2 Round 3 Score: {sa_score3:.6f}")
print(f"Total improvement from baseline: {score - sa_score3:.6f}")

SA Fast V2 Round 3 Score: 70.620111
Total improvement from baseline: 0.004313


In [ ]:
# Check the final score after multiple rounds
final_df = load_submission('/home/code/submission2.csv')
final_score, final_sides = calculate_score(final_df)
print(f"Final SA Fast V2 Score: {final_score:.6f}")
print(f"Total improvement from baseline: {score - final_score:.6f}")

# Validate no overlaps
print("\\nValidating no overlaps...")
has_any_overlap = False
for n in range(1, 201):
    trees = get_trees_for_n(final_df, n)
    has_overlap, overlap_pairs = check_overlaps(trees)
    if has_overlap:
        print(f"  n={n}: OVERLAPS FOUND - {len(overlap_pairs)} pairs")
        has_any_overlap = True

if not has_any_overlap:
    print("All configurations valid - no overlaps!")